In [55]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import traceback


In [56]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [57]:
load_dotenv()
api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"]=api_key

In [58]:
llm=HuggingFaceHub(repo_id="meta-llama/Llama-2-7b-chat-hf",model_kwargs={"temperature": 0.7, "max_length": 512})

In [59]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

import PyPDF2

In [60]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above test, it is your job to \
Create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the test as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [61]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [62]:
prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [63]:
chain=LLMChain(llm=llm,prompt=prompt,output_key="quiz")

In [64]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [65]:
prompt2=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [66]:
chain2=LLMChain(llm=llm,prompt=prompt2,output_key="review")

In [82]:
generate_chain=SequentialChain(chains=[chain,chain2],input_variables=['text','number','subject','tone','response_json'],output_variables=['quiz','review'])

In [83]:
file_path="MCQ_Generator\\data.txt"

In [84]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [85]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.\n\nFrom a theoretical viewpoint, probably approximately correc

In [86]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [87]:
NUMBER=5
SUBJECT="machine learning"
TONE='simple'

In [88]:
response=generate_chain(
    {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)

c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in ht

In [89]:
print(response['quiz'])


Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.

From a theoretical viewpoint, probably approximately correct

In [90]:
import re

match = re.search(r"### RESPONSE_JSON\s*(\{.*\})", response['quiz'], re.DOTALL)
#quiz=json.loads(str(match.group(1)))
response_text=str(match.group(1))
match = re.search(r"###\s*\w+:\s*\n(.*)", response_text, re.DOTALL)
quiz=json.loads(str(match.group(1)))

In [91]:
quiz

{'1': {'mcq': "Who coined the term 'machine learning'?",
  'options': {'a': 'Alan Turing',
   'b': 'Arthur Samuel',
   'c': 'Donald Hebb',
   'd': 'Warren McCulloch'},
  'correct': 'b'},
 '2': {'mcq': 'What is the main goal of machine learning?',
  'options': {'a': 'To create human-like consciousness',
   'b': 'To learn from data and make predictions or decisions',
   'c': 'To replace all human jobs',
   'd': 'To understand the human brain'},
  'correct': 'b'},
 '3': {'mcq': 'Which of the following is a key component of machine learning?',
  'options': {'a': 'Common sense',
   'b': 'Statistical algorithms',
   'c': 'Perfect accuracy',
   'd': 'Human emotions'},
  'correct': 'b'},
 '4': {'mcq': 'What is the primary objective of supervised learning?',
  'options': {'a': 'To find patterns in unlabeled data',
   'b': 'To make predictions using labeled data',
   'c': 'To mimic human creativity',
   'd': 'To perform tasks without any instructions'},
  'correct': 'b'},
 '5': {'mcq': 'Which of

In [92]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq=value['mcq']
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [93]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a:Alan Turing | b:Arthur Samuel | c:Donald Hebb | d:Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': 'What is the main goal of machine learning?',
  'Choices': 'a:To create human-like consciousness | b:To learn from data and make predictions or decisions | c:To replace all human jobs | d:To understand the human brain',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is a key component of machine learning?',
  'Choices': 'a:Common sense | b:Statistical algorithms | c:Perfect accuracy | d:Human emotions',
  'Correct': 'b'},
 {'MCQ': 'What is the primary objective of supervised learning?',
  'Choices': 'a:To find patterns in unlabeled data | b:To make predictions using labeled data | c:To mimic human creativity | d:To perform tasks without any instructions',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is a field related to machine learning?',
  'Choices': 'a:Astrology | b:Data mining | c:Alchemy | d:Homeopathy',
 

In [94]:
quiz_df=pd.DataFrame(quiz_table_data)

In [95]:
quiz_df

,MCQ,Choices,Correct
0,Who coined the term 'machine learning'?,a:Alan Turing | b:Arthur Samuel | c:Donald Heb...,b
1,What is the main goal of machine learning?,a:To create human-like consciousness | b:To le...,b
2,Which of the following is a key component of m...,a:Common sense | b:Statistical algorithms | c:...,b
3,What is the primary objective of supervised le...,a:To find patterns in unlabeled data | b:To ma...,b
4,Which of the following is a field related to m...,a:Astrology | b:Data mining | c:Alchemy | d:Ho...,b


In [96]:
quiz_df.to_csv("Machine_learning.csv",index=False)

In [98]:
print(response['review'])


You are an expert english grammarian and writer. Given a Multiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural la

In [102]:
t="Check from an expert English Writer of the above quiz:"
match=re.search(re.escape(t) + r"\s*(.*)",response['review'],re.DOTALL)
match.group(1)

1. Who coined the term 'machine learning'?
2. What is the main goal of machine learning?
3. Which of the following is a key component of machine learning?
4. What is the primary objective of supervised learning?
5. Which of the following is a field related to machine learning?
